In [26]:
import cv2
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image
import pytesseract
import pandas as pd
import re
import numpy as np
from scipy.ndimage import rotate
import pytesseract
from scipy.ndimage import rotate
import csv

In [37]:
# Устанавливаем путь к исполняемому файлу Tesseract-OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Определяем функцию, которая определит лучший угол поворота изображения для улучшения результатов OCR
def determine_best_rotation_angle(image, lang='rus+eng', rotation_range=(-10, 10), steps=1):
    current_max_confidence = -np.inf # Инициализируем текущий максимум средней уверенности
    best_rotation_angle = 0 # Инициализируем лучший угол поворота
    # Перебираем углы поворота в заданном диапазоне
    for rotation_angle in range(rotation_range[0], rotation_range[1] + steps, steps):
         # Поворачиваем изображение на заданный угол без изменения размера
        rotated_image = rotate(image, rotation_angle, reshape=False)
        # Получаем данные OCR для повернутого изображения
        configs_data = pytesseract.image_to_data(rotated_image, output_type=pytesseract.Output.DICT, lang=lang, config='--psm 6')
        # Рассчитываем среднее значение уверенности OCR в распознавании символов
        mean_confidence = np.mean(np.asarray(configs_data["conf"], dtype=float))
        # Если текущая средняя уверенность больше предыдущей, обновляем лучший угол поворота и текущую максимальную уверенность
        if mean_confidence > current_max_confidence:
            best_rotation_angle = rotation_angle
            current_max_confidence = mean_confidence
    # Возвращаем лучший угол поворота
    return best_rotation_angle

# Задаем путь к папке с изображениями
folder_path = 'datasets/napoleon_it_retail/not_analogues'
files = os.listdir(folder_path) # Получаем список файлов в папке

# Создаем и открываем файл CSV для записи результатов
with open('not_analogues.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['file_name', 'text']) # Записываем строку с заголовками столбцов

# Перебираем файлы в папке и обрабатываем каждое изображение
for file in files:
    image_path = os.path.join(folder_path, file)  # Формируем путь к файлу изображения
    image = cv2.imread(image_path)  # Читаем изображение

    if image is None:  # Если изображение не удалось прочитать, записываем имя файла и пустую строку
        print(f"Unable to read image: {file}")
        csv_writer.writerow([file, ""])
        continue

    # Задаем размеры изображения для уменьшения и рассчитываем коэффициент масштабирования
    target_width = 1600
    target_height = 1600
    scale_factor = min(target_width / image.shape[1], target_height / image.shape[0])
    new_width = int(image.shape[1] * scale_factor)
    new_height = int(image.shape[0] * scale_factor)

    # Изменяем размер изображения с помощью интерполяции
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # Конвертируем цветное изображение в оттенки серого
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Применяем размытие по Гауссу для уменьшения шума
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Нормализуем уровень яркости изображения
    normalized_image = cv2.normalize(blurred_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Определяем лучший угол поворота
    best_rotation_angle = determine_best_rotation_angle(normalized_image)

    # Поворачиваем изображение на лучший угол поворота
    rotated_image = rotate(normalized_image, best_rotation_angle, reshape=False)

    # Обрезаем верхнюю часть изображения (35%)
    cropped_image = rotated_image[:int(rotated_image.shape[0] * 0.35), :]

    # Применяем пороговое преобразование Оцу для бинаризации изображения
    _, binary_image = cv2.threshold(cropped_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Применяем морфологическое преобразование (Дилатация, Замыкание)
    kernel = np.ones((3, 3), np.uint8)
    dilated_image = cv2.dilate(binary_image, kernel, iterations=1)
    morphed_image = cv2.erode(dilated_image, kernel, iterations=1)

    # Указываем параметры OCR
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(morphed_image, config=custom_config, lang='rus+eng')

    # Заменяем символы новой строки пробелом
    text = text.replace('\n', ' ')
    print(text)

    # Записываем имя файла и распознанный текст в файл CSV
    csv_writer.writerow([file, text])

In [33]:
# Считываем csv файл и сохраняем в формате xlsx
import pandas as pd
from string import punctuation, ascii_letters, digits, whitespace
import xlsxwriter

filename_war = 'not_analogues_clear.csv'
data = pd.read_csv(filename_war)

cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ."

def strip(text):
    pnt = punctuation.index('.')
    allowed_chars = cyrillic_letters + ascii_letters + digits + whitespace + punctuation[pnt]
    text = "".join([c for c in text if c in allowed_chars])
    return text

data['text_clean'] = data['text'].apply(lambda x: strip(x))

output_filename = 'not_analogues_clear.xlsx'
workbook = xlsxwriter.Workbook(output_filename, {'strings_to_urls': False, 'strings_to_formulas': False})
worksheet = workbook.add_worksheet()
header_format = workbook.add_format({'bold': True})

# Читаем строку заголовка
for idx, header in enumerate(data.columns):
    worksheet.write(0, idx, header, header_format)

# Читаем данные
for row_idx, row in data.iterrows():
    for col_idx, value in enumerate(row):
        worksheet.write(row_idx + 1, col_idx, value)

workbook.close()

,file_name,text,text_clean
0,00c7f210-035a-4e57-ba49-6a643d07b52e.jpg,"REDDS ПИВНОЙ НАПИТОЖ 0,33Л CT/ BYT",REDDS ПИВНОЙ НАПИТОЖ 033Л CT BYT
1,00f4a973-06b5-4198-bcd1-0f15005165b2.jpg,"53M ж Сыр НОСН!_А!\!!_) sg, B",53M ж Сыр НОСНА sg B
2,00fe5b11-2102-4661-9306-16dba039ef59.jpg,’‚. т mary mMEBCKOP i ) yaee |.,. т mary mMEBCKOP i yaee .
3,010c29a1-b6c5-4455-8470-cc49555263e7.jpg,| ИМУНЕЛЕ НЕО НАПИТОК NPORYKT БЕЗ В | КИсЛОМОЛ...,ИМУНЕЛЕ НЕО НАПИТОК NPORYKT БЕЗ В КИсЛОМОЛОЧ...
4,014760b3-dbe2-4fa2-85fe-a48ad24f46eb.jpg,"tes 5 Lt BETOE 0.44n | РЕа oy by Gy, 5",tes 5 Lt BETOE 0.44n РЕа oy by Gy 5
...,...,...,...
2084,ff992beb-be68-48d2-b2e2-80131ab7477b.jpg,HEKTAP RICH ВИШНЕВЫЙ Be OCBFTNFHEE IS; ‚,HEKTAP RICH ВИШНЕВЫЙ Be OCBFTNFHEE IS
2085,ffb38fd1-8839-46d4-8898-9b8cd8c6a8ca.jpg,"‚ Ботоныик SNICKERS 50,5r Ю",Ботоныик SNICKERS 505r Ю
2086,ffb4c392-b523-49a3-9bce-45819814c489.jpg,|'“… КУККНИ COK ABNOKO/ ГРУША | ОСВЕТ С SMEC 0...,КУККНИ COK ABNOKO ГРУША ОСВЕТ С SMEC 021 1 T...
2087,fff64dd9-81a4-45f1-8a46-4571b8c76a91.jpg,B3M)K СЫР ВИОЛА/ВАЛИО ТВОРОЖНЫЙ СсЛИВОЧчНЫЙ 7...,B3MK СЫР ВИОЛАВАЛИО ТВОРОЖНЫЙ СсЛИВОЧчНЫЙ 70 ...


In [ ]:
import pandas as pd

# Считываем данные из xlsx файла
df = pd.read_excel('output_clean_files/not_analogues_clear.xlsx', 'Sheet1')

# Сохраняем данные в csv файл с кодировкой utf-8-sig
df.to_csv('output_clean_files/not_analogues_clean.csv', encoding='utf-8-sig', index=False)

In [ ]:
import pandas as pd
import os

# Указываем путь к каталогу с jpg файлами
jpg_folder = 'output_clean_files/not_analogues'

# загрузка csv файла в pandas dataframe
df = pd.read_csv('output_clean_files/not_analogues_clean.csv')

# фильтрация строк с attribute==1
df_flag_1 = df[df['attribute'] == 1]

# удаление файлов с attribute==0
for filename in df[df['attribute'] == 0]['file_name']:
    file_path = os.path.join(jpg_folder, filename)  # Получаем полный путь к файлу
    if os.path.exists(file_path) and os.path.isfile(file_path):
        os.remove(file_path)

# Удаление первой колонки из DataFrame (axis=1 указывает, что удаляем колонку)
first_column_name = df_flag_1.columns[0]
df_flag_1 = df_flag_1.drop(first_column_name, axis=1)

# Удаление колонки 'attribute' из DataFrame (axis=1 указывает, что удаляем колонку)
df_flag_1 = df_flag_1.drop('attribute', axis=1)

# сохранение обновленного dataframe в csv файл
df_flag_1.to_csv('output_clean_files/not_analogues_clean_full.csv', index=False, encoding='utf-8-sig')
df_flag_1